<h2>Step 1: Notebook Set-up<h2> 

In [ ]:
# This is just a little piece of code to increase the width of the cells in the notebook.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

<h2>Step 2: Define Your Business Question</h2>
<h4>What defines success? What are you trying to do?</h4>
<p>For today's tutorial, we are going to use the <b><a href=https://datacatalog.worldbank.org/dataset/world-development-indicators>World Development Indicators dataset</a></b>, which presents "the most current and accurate global development data available, and includes national, regional, and global estimates."</p>
<br>
<p>While there are many questions we could answer with this data, today we are going to focus on one question in particular. How hard is it to start a business in different countries?</p>
<br>
<p>That's a pretty vague question, so let's try to refine it further. It costs time and money to start a business, and those two variables are almost certainly correlated. Other factors that may affect the ability to start a business include the regulatory environment and the general level of prosperity in the country. To be specific, let's try to <b>predict the cost of starting a business</b> in countries around the world, given some information about that country.</p>
<br>
<p>This is a problem that is an example of supervised learning: using a data set that contains <b>training examples</b> and their associated correct <b>labels</b> to train a learning algorithm. When the training process is finished, we will have a learned function that can be used to estimate the cost of starting a business for <b>unlabeled test data</b>. E.g. Would it cost more, or less, to start a business if the gross national product in my country went up?</p>
 <br>
<p>Today's tutorial is on a classic regression problem. We are predicting a <b>continuous target variable</b> (cost to start a business) based on <b>input features</b> (regulatory environment, number of people). </p>

<h2>Step 3: Collect Your Data</h2>
<p>Here we are importing data from CSV files into <b>Pandas dataframes</b>.</p>
<p>"Pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."</p>
<p><a href:"https://pandas.pydata.org/">Pandas documentation</a></p>

In [ ]:
data_dir = '.'

In [ ]:
business_data_file = 'CombinedStartupCellularData.csv'
country_data_file = 'CountryDimData.csv'
regional_cellular_data_file = 'RegionalCellularData.csv'
country_cellular_data_file = 'CountryCellularData.csv'

In [ ]:
business_df = pd.read_csv(os.path.join(data_dir, business_data_file))
country_df = pd.read_csv(os.path.join(data_dir, country_data_file))
r_cellular_df = pd.read_csv(os.path.join(data_dir, regional_cellular_data_file))
c_cellular_df = pd.read_csv(os.path.join(data_dir, country_cellular_data_file))

<h3>A Little About These Data</h3>
<p>These are prepared data files, similar to those created during the "Data Engineering" tutorial.
<ul>
    <li>business_df (by country): startup cost, startup time, ease of business, population, broadband subscriptions, cell subscriptions, percentage of population over age 65, GNI per capita
    <li>country_df (by country): name, region, and income group for each country
    <li>r_cellular_df (by region and year): population and number of broadband and cellular subscriptions
    <li>c_cellular_df (by country and year): population and number of broadband and cellular subscriptions
</ul>
<p><b>Exercise:</b> Look at the head of each of the four dataframes.

In [ ]:
business_df.head()

<h2>Step 4: Data Exploration</h2>
<p>The beginning of any data science project - take a look at your data. Pay attention to missing values, categorical vs. continuous variables, data size. Your analysis or model will only ever be as good as the data that goes into it. 

<h4>How big is the data?</h4>
<p><b>Exercise:</b> Look at the sizes of each dataframe

In [ ]:
business_df.shape
# c_cellular_df.shape

<h4>What does it look like?</h4>

In [ ]:
c_cellular_df.sample(10)

<h4>How is the data distributed?</h4>

In [ ]:
business_df.describe()

<h4>Are there nulls in the data?</h4>
<p><b>Exercise</b>: How many nulls are in the country and cellular data sets? What are they?</p>

In [ ]:
business_df.isna().sum()

<h4>If there are nulls, what countries do they affect?</h4>

In [ ]:
business_df[business_df['ease_of_business'].isnull() == True].sample(10)

<h2>Step 5: Dealing with missing values</h2>
<p>Most data sets will have missing values. How you decide to impute those values (or whether you choose to drop those rows) can make a big difference in your results.
<p>Two simple ways to impute data are shown in the cell below. In the first, we fill in all missing values with zero. In the second, we fill in all missing values with the mean of the other values in that column.
<p><b>Discussion:</b> What are the effects of filling with zero vs. filling with the mean? Is there a "smarter" way to impute missing data?
<p><b>Exercise:</b> Try this with one of the cellular data sets.

In [ ]:
business_df_fill_zero = business_df.fillna(0)
business_df_fill_mean = business_df.fillna(business_df.mean())

<h4>Impute with mean by income group.</h4>
<p>Here we use the country data (country_df) to do a "smart" mean where missing values get filled by the mean for countries in the same income group.</p>
<p><b>Note:</b> This section uses a lot of Pandas functions and the syntax isn't straightforward if you are relatively new to Pandas. If you don't understand what's happening in these cells, that's okay. Simply run them and circle back to understanding them alter.</p>
<p><b>Exercise:</b> Try to walk through each step and make sure you understand what's happening.

In [ ]:
# Get the column names for the business dataframe
original_columns = business_df.columns

In [ ]:
# Create a combined data frame that includes the business data, as well as the income group for each country
df_join = pd.merge(business_df, country_df[['country_name', 'income_group']], on='country_name')

In [ ]:
# Group by the income group, then take the mean. We reset the index for ease of use.
df_grouped = df_join.groupby('income_group').mean().reset_index()

In [ ]:
df_grouped.head()

In [ ]:
# Rename the columns of the grouped data frame for clarity. (They will ordinarily have the same names as the ungrouped dataframe and that would be confusing.)
df_grouped.columns = ['income_group', 'mean_startup_cost_pct_gni', 'mean_startup_time', 'mean_startup_procedures', 'mean_ease_of_business', 'mean_population', 'mean_broadband_subscriptions', 'mean_cellular_subscriptions', \
                      'mean_percentage_pop_age_65_and_above', 'mean_gni_per_capita']

In [ ]:
# Join the grouped data frame back to our business+country dataframe.
df_join2 = pd.merge(df_join, df_grouped, on='income_group')

In [ ]:
# Finally, fill in the missing values with the mean calculated by income group.
df_join2['ease_of_business'].fillna(df_join2['mean_ease_of_business'], inplace=True)
df_join2['broadband_subscriptions'].fillna(df_join2['mean_broadband_subscriptions'], inplace=True)
df_join2['cellular_subscriptions'].fillna(df_join2['mean_cellular_subscriptions'], inplace=True)
df_join2['percentage_pop_age_65_and_above'].fillna(df_join2['mean_percentage_pop_age_65_and_above'], inplace=True)


In [ ]:
business_df_fill_smart = df_join2[original_columns]
business_df_fill_smart.fillna(business_df_fill_smart.mean(), inplace=True)

In [ ]:
business_df_fill_mean.sample(5)

<h2>Step 6: Pair Plots: Useful technique to understand your variables</h2>
<p>We've spent a lot of time looking at tables of our data, but sometimes the best and easiest way to understand our data is simply to look at it. Here we use pair-plots to quickly understand the underlying data.
<p><b>What is a pair plot? (aka scatterplot matrix)</b></p>
<p>A bunch plots showing each variable in the data set plotted against every other variable in the data set. Usually a histogram of the variable runs down the diagonal.</p>
<p><a href=https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166>A useful blog post.</a>
<p><b>Exercise:</b> Identify, by eye, variables that are colinear. Which do you think are the most predictive?
<p><b>Exercise:</b> Try this scatter plot with business_df_fill_zero, business_df_fill_smart where the variables have been filled in different ways.

In [ ]:
# create a scatter matrix from the dataframe, color by y_train
Y = np.array(business_df_fill_smart['startup_cost_pct_gni_per_capita'])
grr = pd.plotting.scatter_matrix(business_df_fill_smart.drop(['country_name'], axis=1), c=Y,figsize=(16, 16), marker='o',
                                 hist_kwds={'bins': 50}, s=60, alpha=.8)
n = len(business_df_fill_smart.columns)-1
for x in range(n):
    for y in range(n):
        # to get the axis of subplots
        ax = grr[x, y]
        # to make x axis name vertical  
        ax.xaxis.label.set_rotation(90)
        # to make y axis name horizontal 
        ax.yaxis.label.set_rotation(0)
        # to make sure y axis names are outside the plot area
        ax.yaxis.labelpad = 100

<h2>Step 7: Get a Baseline Result</h2>
<p>We've looked at our data and filled in missing values. We've decided on a target variable (Startup Cost as a Percentage of GNI). Now it's time to get to building our predictive models.</p>
<p>It's often a good idea to try a simple model(e.g. linear or logistic regression) as your first step, as it's possible that you don't need anything fancier. If you do, benchmarking against the linear regression can be a useful measure.</p>
<br><p><b>Linear regression</b>, also known as ordinary least squares or "drawing the best fit line" is a parametric method, meaning that we assume a form for the function that relates our input features to our target variable.</p>
<p><tt>y = M*x+b</tt></p>
<br>
<p><b>Exercise:</b> Try the linear model and random forest model with one of the other data sets.

<h4>Step 7a: Train/test/validation</h4>
<p>We need to split our data set into <b>training</b> and <b>testing</b> sets. The training set will be used to train the model, while the testing set will be used to get an accurate representation of how well the model predicts data it has never seen before. This is done to prevent <b><a href=https://en.wikipedia.org/wiki/Overfitting>overfitting</a></b>.</p>

In [ ]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.loc[perm[:train_end]]
    validate = df.loc[perm[train_end:validate_end]]
    test = df.loc[perm[validate_end:]]
    return train, validate, test

In [ ]:
# Note that this fuction includes the ability to split into training, testing, and validation sets. Here we will ignore the validation set and set that percentage to zero.
train, validate, test = train_validate_test_split(business_df_fill_mean, train_percent=0.8,validate_percent = 0)

In [ ]:
print('There are '+ str(len(train)) + ' data points in the training set.')
print('There are '+ str(len(test)) + ' data points in the testing set.')

In [ ]:
train.head()

<h4>Step 7b: Set-up the data for training and evaluation.</h4>
<p>We need to split out our target variable (y) and transform our dataframes into matrices.</p>

In [ ]:
y = np.array(train['startup_cost_pct_gni_per_capita'])
X = np.array(train.drop(['country_name', 'startup_cost_pct_gni_per_capita'], axis=1))
y_test = np.array(test['startup_cost_pct_gni_per_capita'])
X_test = np.array(test.drop(['country_name', 'startup_cost_pct_gni_per_capita'], axis=1))

<h2>Step 8: Linear Regression</h2>
<p>Now, in suprisingly few lines, we can create and train/fit our linear regression. We also get the predictions of the fitted model on the test set, to compare against the known (correct) labels.</p>

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X,y)
y_pred = reg.predict(X)
y_test_pred = reg.predict(X_test) #Look at the test set (holdout)

<h2>Step 9: Look at the Regression Coefficients</h2>
<p>One of the advantages of linear regression is that the coefficients learned by the model are extremely interpretable and have a clear meaning. Here, we map the learned coefficients back to their column names and take a look at the results.</p>

In [ ]:
# input_feature_names = ['Startup Time', 'Startup Procedures', 'GNI', 'GDP Per Capita', 'Business Regulation', 'Ease of business']
input_feature_names = train.drop(['country_name', 'startup_cost_pct_gni_per_capita'], axis=1).columns

coef_dict = {input_feature_names[i]: reg.coef_[i] for i in range(0, len(input_feature_names))}

In [ ]:
plt.figure(figsize=(8,10))
plt.barh(range(len(coef_dict)), list(coef_dict.values()), align='center', tick_label=list(coef_dict.keys()))
plt.xlabel('Fitted Coefficient')
plt.axvline(x=0)

<h2>Step 10: Model Error </h2>
<p>Here we look at the model error in two ways. In the first, we do a visual examination of the model's predictions on both the training and test sets and compare them the actual values. If the model were absolutely perfect, every point in these plots would lie along the <tt>x=y</tt> line.</p>
<p>The second method of examinining the model error is via the RMSE (root mean squared error).</p>
<p>Add sentence here about what the error is.</p>
<p>Examples: Predicting home prices. Stock market prediction. Content demand prediction.</p>

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.plot(y, y_pred, '.')
plt.plot([min(y), max(y)], [min(y), max(y)])
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Training Set')

plt.subplot(1,2,2)
plt.plot(y_test, y_test_pred, '.')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)])
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Test Set')

In [ ]:
from sklearn.metrics import mean_squared_error
print('Root mean squared error for training set: ' + str(round(np.sqrt(mean_squared_error(y, y_pred)))))
print('Root mean squared error for test set: ' + str(round(np.sqrt(mean_squared_error(y_test, y_test_pred)))))

<h2>Step 11: Iterate and Experiment on Your Model</h2>
<p>Now we can move on to trying other ML techniques and see if our predictions improve. Here we are trying the random forest algorithm, a nonparametric approach that uses ensembles of decision trees to make predictions. They tend to be a common "go-to" technique for supervised ML problems, since they are robust to data that hasn't been standardized or that isn't super-clean.</p>
<p>Scikit-learn (and other associated ML packages) make trying various algorithms extremely easy. However, throwing the kitchen sink at your problem will often result in "gotchas". That is, just trying every algorithm and picking the best one is not the right thing to do. Think about what algorithm suits your problem and your data.</p>

<h4>Random Forest</h4>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=0)
rf_reg.fit(X,y)
y_pred_rf = rf_reg.predict(X)
y_test_pred_rf = rf_reg.predict(X_test)

<h2>Step 12: Look at feature importances</h2>
<p>Feature importances are calculated in tree-based models. It's useful to look at them to see which of our features are most relevant/helpful to the model.</p>

In [ ]:
input_feature_names = train.drop(['country_name', 'startup_cost_pct_gni_per_capita'], axis=1).columns
fe_dict = {input_feature_names[i]: rf_reg.feature_importances_[i] for i in range(0, len(input_feature_names))}

In [ ]:
plt.figure(figsize=(8,10))
plt.barh(range(len(fe_dict)), list(fe_dict.values()), align='center', tick_label=list(coef_dict.keys()))
plt.xlabel('Feature Importance')
plt.axvline(x=0)

<h2>Step 13: Model Error</h2>
<p>It will be instructive to compare the model error from the random forest model to the model error from the linear regression. In particular, check whether the training and test errors are both improving. If the training error is improving, but the test error is not, you may be overfitting.</p>

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plt.plot(y, y_pred_rf, '.')
plt.plot([min(y), max(y)], [min(y), max(y)])
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Training Set')

plt.subplot(1,2,2)
plt.plot(y_test, y_test_pred_rf, '.')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)])
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Test Set')

In [ ]:
print('Root mean squared error for training set: ' + str(round(np.sqrt(mean_squared_error(y, y_pred_rf)))))
print('Root mean squared error for test set: ' + str(round(np.sqrt(mean_squared_error(y_test, y_test_pred_rf)))))